In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import os

In [5]:
a = torch.zeros([1,5,5])

a = torch.stack([a]*5,dim=0)

print a.shape

torch.Size([5, 1, 5, 5])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,padding=(2,2))
        self.conv2 = nn.Conv2d(6,16,5,padding=(2,2))
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*7*7,1024)
        self.fc2 = nn.Linear(1024,84)
        self.fc3 = nn.Linear(84,10)
    
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = self.conv2(x)
        x = self.pool(F.relu(x))
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = F.softmax(x)
        return x
    

if __name__=="__main__":
    net = Net()
    print net  


In [40]:
class CapsNet(nn.Module):
    """
    input :a group of capsule -> shape:[batch_size*1152(feature_num)*8(in_dim)]
    output:a group of new capsule -> shape[batch_size*10(feature_num)*16(out_dim)]
    """
    
    def __init__(self,in_features,out_features,in_dim,out_dim):
        """
        """
        super(Capsnet,self).__init__()
        #number of output features,10
        self.out_features = out_features
        #number of input features,1152
        self.in_features = in_features
        #dimension of input capsule
        self.in_dim = in_dim
        #dimension of output capsule
        self.out_dim = out_dim
        
        #full connect parameter W with shape [1(batch共享),1152,10,8,16]
        self.W = nn.Parameter(torch.randn(1,self.in_features,self.out_features,in_dim,out_dim))
        
    def forward(self,x):
        #input x,shape=[batch_size,in_features,in_dim]
        #[batch_size,1152,8]
        # (batch, input_features, in_dim) -> (batch, in_features, out_features,1,in_dim)
        x = torch.stack([x] * self.out_features, dim=2).unsqueeze(3)
        # u_hat shape->(batch_size,in_features,out_features,out_dim)=(batch,1152,10,16,1)
        u_hat = torch.matmul(self.W,x)
        #b for generate weight c,with shape->[1,1152,10,1]
        b = torch.zeros([1,self.in_features,self.out_features,1])
        for i in range(3):
            c = F.softmax(b,dim=2)
            #c shape->[batch_size,1152,10,1,1]
            c = torch.cat([c] * batch_size, dim=0).unsqueeze(dim=4)
            #s shape->[batch_size,1,10,16,1]
            s = u_hat * c.sum(dim=1,keep_dim=True)
            #output shape->[batch_size,1,10,16,1]
            v = squash(s,dim=-2)
            v_1 = torch.cat([v] * self.in_features, dim=1)
            #(batch,1152,10,1,16)matmul(batch,1152,10,16,1)->(batch,1152,10,1,1)
            #squeeze
            #mean->(1,1152,10,1)
            update_b = torch.matmul(u_hat.transpose(3,4),v_1).squeeze(dim=4).mean(dim=0,keep_dim=True)
            b = b+update_b
        return v.squeeze(dim=1)
    

if __name__ == "__main__":
    net = CapsNet()
    print net
    

TypeError: __init__() takes exactly 5 arguments (1 given)

In [ ]:
class CapsConv(nn.Module):
    def __init__(self,in_channels=256,out_dim=8):
        self.in_channels = in_channels
        self.out_dim = out_dim
        super(CapsConv,self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=32,  # fixme constant
                              kernel_size=9,  # fixme constant
                              stride=2, # fixme constant
                              bias=True)
        
    def forward(self,x)
        #input x with shape ->[batch_size,in_features,height,width]
        #output with shape->[batch_size,32,6,6]
        x = [self.conv(x) for i in range(self.out_dim)]
        #output with shape->[batch_size,8,32,6,6]
        x = torch.stack(x,dim=1)
        #return shape->[batch_size,1152,8]
        x = squash(x,dim=2)
        return x.view(x.size(0),self.out_dim,-1).transpose(1,2)
    

In [21]:
def squash(x,dim):
    #we should do spuash in each capsule.
    #we use dim to select
    sum_sq = torch.sum(x**2,dim=dim,keep_dim=True)
    sum_sqrt = torch.sqrt(sum_sq)
    return (sum_sq/(1.0+sum_sq))* x/sum_sqrt
    
def loss(labels,v):
    """
    input:
        labels:[batch_size,10]
        v:[batch_size,10,16,1]
    """
    #shape->[batch_size,10,1,1]
    v_norm = torch.sqrt(torch.sum(v**2,dim=2,keep_dim=True)).squeeze()
    zero = torch.zeros([1])
    m_plus = 0.9
    m_minus = 0.1
    lamda = 0.5
    
    #shape->[batch_size,10]
    L = torch.max(0,m_plus-v_norm)**2
    R = torch.max(0,v_norm-m_minus)**2
    #equation 4 in paper
    loss = torch.sum(labels*L+(1-labels)*R,dim=1)
    #shape->[batch_size,]
    loss = loss.mean()
    #shape->[1,]
    return loss

In [ ]:
a = np.ones(shape=[1152,1,8])
weight = np.ones(shape=[10,1152,8,16])
u = np.zeros(shape=[10,1152,1,16])
c = np.zeros(shape=[10,1152])
s = np.zeros(shape=[10,16])


for i in range(weight.shape[0]):
    for j in range(a.shape[0]):
        u[i][j]=a[i].dot(weight[i][j])
    s[i] = np.sum(c[i]*s[i].squeeze(),axis=0)

print s.shape



def spuashing(s):
    return s
 
def dynamic_routing(d,r=3):
    b = np.ones(shape=[1152,10])
    for iteration in range(r):
        #在行上面做softmax，
        #(output_dim)
        c = F.softmax(b,axis=0)
        v = spuashing(s)
        b = b+u*v
    

In [17]:
a = np.ones(shape=[3,3])
b = np.ones(shape=[3,8])

print np.matmul(a,b).shape
a = torch.Tensor(b)
a = Variable(a)
F.softmax(a,dim=0)


(3, 8)


Variable containing:
 0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333
 0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333
 0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333  0.3333
[torch.FloatTensor of size 3x8]

In [11]:
a = [torch.zeros([256,32,6,6]) for i in range(8)]

b = torch.stack(a,dim=1)
print b.shape


torch.Size([256, 8, 32, 6, 6])


In [14]:
a = torch.ones([1,1152,10,1,8])

b = torch.ones([1,1152,10,8,16])

c = torch.matmul(a,b)
print c.shape

torch.Size([1, 1152, 10, 1, 16])


In [33]:
a = torch.randn([4,5])
b = torch.ones([5,6,5])
zero = torch.zeros([1])

print torch.max(a,zero)

torch.sum(a,dim=1).mean()


 1.5112  0.7516  0.5362  2.7552  0.0000
 0.7594  0.0000  0.0000  0.1409  0.3340
 0.7613  0.0000  0.1501  0.0000  0.0280
 0.0000  0.1042  3.3040  1.0825  0.6528
[torch.FloatTensor of size 4x5]



1.8632503002882004

In [20]:
a = torch.ones([5,5,6])

b = a.unsqueeze(dim=3)
print b.shape

torch.Size([5, 5, 6, 1])


In [36]:
for i in range(3):
    print i

0
1
2
